In [ ]:
from typing import List

import missingno as msno
import pandas as pd
from IPython.display import HTML
import numpy as np

from sklearn.impute import SimpleImputer


In [ ]:
CATEGORICAL_FEATURES = [
    "url",
    "code",
    "creator",
    "created_datetime",
    "last_modified_datetime",
    "product_name",
    "generic_name",
    "packaging",
    "packaging_tags",
    "brands",
    "brands_tags",
    "categories",
    "categories_tags",
    "categories_fr",
    "origins",
    "origins_tags",
    "manufacturing_places",
    "manufacturing_places_tags",
    "labels",
    "labels_tags",
    "labels_fr",
    "emb_codes",
    "emb_codes_tags",
    "first_packaging_code_geo",
    "cities",
    "cities_tags",
    "purchase_places",
    "stores",
    "countries",
    "countries_tags",
    "countries_fr",
    "ingredients_text",
    "traces",
    "traces_tags",
    "no_nutriments",
    "additives",
    "additives_tags",
    "ingredients_from_palm_oil",
    "ingredients_from_palm_oil_tags",
    "ingredients_that_may_be_from_palm_oil",
    "ingredients_that_may_be_from_palm_oil_tags",
    "nutrition_grade_fr",
    "nutrition_grade_uk",
    "main_category",
    "main_category_fr",
    "image_url",
    "image_small_url",
    "allergens",
    "allergens_fr",
    "traces_fr",
    "additives_fr",
    "pnns_groups_1",
    "pnns_groups_2",
    "states",
    "states_tags",
    "states_fr",
]


In [ ]:
foods = pd.read_csv(
    "files/fr.openfoodfacts.org.products.csv", sep="\t", low_memory=False
)


## Clean and filter features and product

In [ ]:
def clean_and_filter_features_and_product(
    df: pd.DataFrame, feature: str, sub_features=List[str]
):
    df[feature] = df[feature].replace(
        {
            "cereals": "Cereals",
            "fruits": "Fruits",
            "legumes": "Legumes",
            "pastries": "Pastries",
            "nuts": "Nuts",
            "vegetables": "Vegetables",
        }
    )

    categorical_foods = foods[CATEGORICAL_FEATURES]

    categorical_foods_mean_null_by_column = categorical_foods.isnull().mean()
    missing_percentage = categorical_foods_mean_null_by_column[feature] * 100

    if missing_percentage < 50:
        return display(
            HTML(
                f"""
                    <p style='color:orange;'>The feature {feature} has less than 50% of missing values ({round(missing_percentage, 1)} %).</p>
                    "<p style='color:red;'>Consider choosing another feature.</p>"
                     <hr style="border:1px solid #000;" />
                """
            )
        )
    display(
        HTML(
            f"""
                <p style='color:green;'>The feature {feature} has more than 50% of missing values ({round(missing_percentage, 1)} %).</p>
                <hr style="border:1px solid #000;" />
            """
        )
    )

    df = df.dropna(subset=[feature])
    df = df[df[feature] != "unknown"]

    sub_features_foods = df[sub_features]

    sub_features_foods_mean_null_by_column = sub_features_foods.isnull().mean()

    at_least_sub_feature_with_missing_values_gt_50_percent = False
    for sub_feature in sub_features:
        missing_percentage = sub_features_foods_mean_null_by_column[sub_feature] * 100

        if missing_percentage > 50:
            at_least_sub_feature_with_missing_values_gt_50_percent = True
            display(
                HTML(
                    f"<p style='color:orange;'>The feature {sub_feature} has more than 50% of missing values ({round(missing_percentage, 1)} %).</p>"
                )
            )
        else:
            display(
                HTML(
                    f"<p style='color:green;'>The feature {sub_feature} has less than 50% of missing values ({round(missing_percentage, 1)} %).</p>"
                )
            )

    if at_least_sub_feature_with_missing_values_gt_50_percent:
        display(
            HTML(
                "<p style='color:red;'>At least one feature has more than 50% of missing values. Consider choosing another feature.</p>"
            )
        )

    sub_features_foods = sub_features_foods.copy()

    sub_features_foods = sub_features_foods.dropna(
        subset=[
            "product_name",
            "brands",
            "packaging",
            "quantity",
            "countries",
            "ingredients_text",
        ]
    )

    sub_features_foods = sub_features_foods.dropna(
        subset=[
            "proteins_100g",
            "carbohydrates_100g",
            "sugars_100g",
            "fat_100g",
            "saturated-fat_100g",
            "salt_100g",
            "sodium_100g",
            "energy_100g",
        ],
        how="all",
    )

    sub_features_foods = sub_features_foods.drop_duplicates(
        subset=[
            "product_name",
            "brands",
            "packaging",
            "quantity",
            "countries",
            "ingredients_text",
            "proteins_100g",
            "carbohydrates_100g",
            "sugars_100g",
            "fat_100g",
            "saturated-fat_100g",
            "salt_100g",
            "sodium_100g",
            "energy_100g",
        ],
        keep=False,
    )
    display(msno.bar(sub_features_foods))
    return sub_features_foods


cleaned_df = clean_and_filter_features_and_product(
    foods,
    "pnns_groups_2",
    [
        "code",
        "product_name",
        "pnns_groups_2",
        "brands",
        "packaging",
        "quantity",
        "countries",
        "ingredients_text",
        "proteins_100g",
        "carbohydrates_100g",
        "sugars_100g",
        "fat_100g",
        "saturated-fat_100g",
        "salt_100g",
        "sodium_100g",
        "energy_100g",
    ],
)


## Manage outliers


In [ ]:
columns_to_check_outliers = [
    "proteins_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fat_100g",
    "saturated-fat_100g",
    "salt_100g",
    "sodium_100g",
]

cleaned_df[columns_to_check_outliers] = cleaned_df[columns_to_check_outliers].where((cleaned_df[columns_to_check_outliers] >= 0) & (cleaned_df[columns_to_check_outliers] <= 100), np.nan)

display(msno.bar(cleaned_df))

copy = cleaned_df.copy()
copy = copy.drop(columns=[
        "packaging",
        "quantity",
        "countries",
        "ingredients_text",
    ],)

copy2 = copy.copy()


groups_of_foods = cleaned_df.pnns_groups_2.unique()
print(groups_of_foods)

for group in groups_of_foods:
    for column in columns_to_check_outliers:
        skewness = cleaned_df[cleaned_df["pnns_groups_2"] == group][column].skew()
        if np.isnan(skewness):
            continue
        elif abs(skewness) >= 1:
            strategy = 'median'
        elif abs(skewness) < 0.5:
            strategy = 'mean'
        else:
            strategy = 'median'

        imputer = SimpleImputer(strategy=strategy)
        group_data = copy2[copy2["pnns_groups_2"] == group]
        imputed_values = imputer.fit_transform(group_data[[column]])
        copy2.loc[copy2['pnns_groups_2'] == group, column] = imputed_values
